In [3]:
import torch
import warnings
from parrot import Parrot
from decouple import config
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from paraphrase import wordtune
warnings.filterwarnings("ignore")

# Parrot Model

In [5]:
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5")

# Sample sentences

In [6]:
phrases = ["I'm looking for lab test detail",
           "Are there any risks to the RBC blood test?",
           "What happens during the Endoscopy?",
           "When should you not take acetaminophen?",
           "What are acetaminophen side-effects?",
           "How should I take acetaminophen?",
           "What other drugs will affect acetaminophen?"
]

# Parrot with do_diverse=False 

In [8]:
for phrase in phrases:
  print("-"*100)
  print("Input_phrase: ", phrase)
  print("-"*100)
  para_phrases = parrot.augment(input_phrase=phrase, 
                                use_gpu=True)
  for para_phrase in para_phrases:
   print(para_phrase[0])

----------------------------------------------------------------------------------------------------
Input_phrase:  I'm looking for lab test detail
----------------------------------------------------------------------------------------------------
i'm looking for the details of the lab tests
i'm looking for details on the lab test
i'm looking for detailed lab test details
i'm looking for a lab test detail
i'm looking for lab test details
i'm looking for lab test detail
----------------------------------------------------------------------------------------------------
Input_phrase:  Are there any risks to the RBC blood test?
----------------------------------------------------------------------------------------------------
is rbc blood test a risk?
is the rbc blood test a dangerous?
are there risks associated with doing an rbc blood test?
do you think there are risk factors in rbc test?
are there any risks in testing for rbc blood?
----------------------------------------------------

# Parrot with do_diverse=True

In [9]:
for phrase in phrases:
  print("-"*100)
  print("Input_phrase: ", phrase)
  print("-"*100)
  para_phrases = parrot.augment(input_phrase=phrase, 
                                use_gpu=True,
                                do_diverse=True)
  for para_phrase in para_phrases:
   print(para_phrase[0])

----------------------------------------------------------------------------------------------------
Input_phrase:  I'm looking for lab test detail
----------------------------------------------------------------------------------------------------
tell me the detail of the lab test
tell me the lab test details
tell me the lab test detail?
tell me the lab test detail
i'm looking for laboratory test details
i'm looking for laboratory test detail
i'm looking for lab test details
i'm looking for lab test detail
----------------------------------------------------------------------------------------------------
Input_phrase:  Are there any risks to the RBC blood test?
----------------------------------------------------------------------------------------------------
do you think the rbc blood test is dangerous?
do you think the rbc blood test is safe?
are there any risks associated with the blood test for rbc?
is there any risk to the blood test for rbc?
are there any risks associated wit

# PEGASUS - pegasus_paraphrase model

In [17]:
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [6]:
def get_response(input_text,num_return_sequences,num_beams):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [16]:
num_beams = 10
num_return_sequences = 8

for phrase in phrases:
  print("-"*100)
  print("Input_phrase: ", phrase)
  print("-"*100)
  para_phrases = get_response(phrase,
                              num_return_sequences,
                              num_beams)
  for para_phrase in para_phrases:
   print(para_phrase)

----------------------------------------------------------------------------------------------------
Input_phrase:  I'm looking for lab test detail
----------------------------------------------------------------------------------------------------
I'm interested in lab test detail.
The lab test detail is something I'm looking for.
I want lab test detail.
I need lab test detail.
I am looking for lab test detail.
I'm looking for lab test details.
I'm looking for the details of the test.
I am looking for lab test details.
----------------------------------------------------------------------------------------------------
Input_phrase:  Are there any risks to the RBC blood test?
----------------------------------------------------------------------------------------------------
Is the blood test safe?
There are risks to the blood test.
Is there any risk to the blood test?
Does the blood test have any risks?
There are no risks to the blood test.
There is a blood test.
Is there a risk to th

# WordTune

In [4]:
token = config('TOKEN')
draftId = config('DRAFTID')
paraphrase = wordtune(token, draftId)

In [8]:
for phrase in phrases:
  print("-"*100)
  print("Input_phrase: ", phrase)
  print("-"*100)
  payload = paraphrase.payload_generator(phrase)
  headers = paraphrase.headers_generator()
  response = paraphrase.requests(payload, headers)
  para_phrases = paraphrase.get_suggestion(response, phrase)
  for para_phrase in para_phrases:
   print(para_phrase)

----------------------------------------------------------------------------------------------------
Input_phrase:  I'm looking for lab test detail
----------------------------------------------------------------------------------------------------
I need lab test details
Details of laboratory tests are needed
I would like to know the details of a lab test
Please provide me with the details of the lab test
I need detailed information about lab tests
I need details about a lab test
The result of my lab test is what I am looking for
I'm interested in details of lab tests
I need details on the lab test
For more details on a lab test, I need them
----------------------------------------------------------------------------------------------------
Input_phrase:  Are there any risks to the RBC blood test?
----------------------------------------------------------------------------------------------------
Are there any risks associated with the RBC blood test
RBC blood tests pose any risks?
Wh